In [10]:
import numpy as np
import scipy.stats as stats
import pandas as pd
from scipy.signal import welch
import mne 
import logging
import os
import warnings

In [11]:
def load_concat(file_paths):
    mean_all= []
    std_all= []
    skew_all= []
    kurtosis_all= []
    for i, file in enumerate(file_paths):
        temp = pd.read_csv(file)
        mean_all.append(temp["mean"])
        std_all.append(temp['std'])
        skew_all.append(temp['skew'])
        kurtosis_all.append(temp['kurtosis'])
    return np.array(mean_all).T,np.array(std_all).T,np.array(skew_all).T,np.array(kurtosis_all).T

In [12]:
x =pd.read_csv("特征\认知正常\时域\余洪涛.csv")
x

,channel,mean,std,skew,kurtosis
0,1,-0.000054,0.001366,-24.718356,613.500031
1,2,-0.000054,0.001367,-24.678128,612.168358
2,3,-0.000054,0.001363,-24.924281,620.339324
3,4,-0.000054,0.001366,-24.749747,614.545384
4,5,-0.000054,0.001362,-24.948666,621.149951
5,6,-0.000054,0.001362,-24.948114,621.131603
6,7,-0.000054,0.001364,-24.845103,617.709333
7,8,-0.000054,0.001364,-24.852634,617.957900
8,9,-0.000054,0.001369,-24.572342,608.662297
9,10,-0.000054,0.001368,-24.643547,611.020571


In [13]:
# 忽略 RuntimeWarning 警告
warnings.filterwarnings("ignore", category=RuntimeWarning)
# 定义文件夹路径
base_dir = '特征'
normal_dir = os.path.join(base_dir, '认知正常','时域')
impaired_dir = os.path.join(base_dir, '认知障碍','时域')

normal_time_files = [os.path.join(normal_dir, f) for f in os.listdir(normal_dir) if f.endswith('.csv')]
impaired_time_files = [os.path.join(impaired_dir, f) for f in os.listdir(impaired_dir) if f.endswith('.csv')]

mean_nc,std_nc,skew_nc,kurtosis_nc = load_concat(normal_time_files)

mean_mci,std_mci,skew_mci,kurtosis_mci = load_concat(impaired_time_files)

In [14]:
# 初始脑区定义
from itertools import combinations
regions = {
    "prefrontal": [0, 1, 2, 3, 10, 11, 16],
    "central": [4, 5, 17],
    "temporal": [12, 13, 14, 15],
    "parietal": [6, 7, 18],
    "occipital": [8, 9]
}

# 自动生成多脑区组合
def generate_combinations(regions, sizes):
    combined_regions = {}
    region_names = list(regions.keys())

    # 遍历指定组合大小
    for size in sizes:
        for combination in combinations(region_names, size):
            combined_name = "_".join(combination)  # 组合名称
            combined_indices = sorted(set().union(*(regions[region] for region in combination)))  # 合并去重
            combined_regions[combined_name] = combined_indices

    return combined_regions

# 生成所有二、三、四脑区组合
regions = generate_combinations(regions, sizes=[1,2, 3, 4,5])


In [15]:
import numpy as np
from scipy import stats

# 假设有两个群体的数据（每行一个样本，每列一个特征）
group_A = std_nc.T  # 群体A有30个样本，5个特征
group_B = std_mci.T  # 群体B有30个样本，5个特征

# 对每个特征进行独立样本t检验
for i in range(group_A.shape[1]):
    t_stat, p_value = stats.ttest_ind(group_A[:, i], group_B[:, i])
    print(f"Feature {i+1}: t-statistic = {t_stat:.3f}, p-value = {p_value:.3f}")


Feature 1: t-statistic = -0.702, p-value = 0.486
Feature 2: t-statistic = -0.728, p-value = 0.470
Feature 3: t-statistic = -0.924, p-value = 0.360
Feature 4: t-statistic = -0.612, p-value = 0.543
Feature 5: t-statistic = -0.551, p-value = 0.584
Feature 6: t-statistic = -0.550, p-value = 0.585
Feature 7: t-statistic = -0.528, p-value = 0.600
Feature 8: t-statistic = -0.599, p-value = 0.552
Feature 9: t-statistic = -0.627, p-value = 0.533
Feature 10: t-statistic = -0.404, p-value = 0.687
Feature 11: t-statistic = -0.613, p-value = 0.543
Feature 12: t-statistic = -1.143, p-value = 0.258
Feature 13: t-statistic = -0.548, p-value = 0.586
Feature 14: t-statistic = -0.614, p-value = 0.542
Feature 15: t-statistic = -0.464, p-value = 0.645
Feature 16: t-statistic = -0.998, p-value = 0.323
Feature 17: t-statistic = -0.543, p-value = 0.589
Feature 18: t-statistic = -0.566, p-value = 0.573
Feature 19: t-statistic = -0.580, p-value = 0.564


In [23]:
regions

{'prefrontal': [0, 1, 2, 3, 10, 11, 16],
 'central': [4, 5, 17],
 'temporal': [12, 13, 14, 15],
 'parietal': [6, 7, 18],
 'occipital': [8, 9],
 'prefrontal_central': [0, 1, 2, 3, 4, 5, 10, 11, 16, 17],
 'prefrontal_temporal': [0, 1, 2, 3, 10, 11, 12, 13, 14, 15, 16],
 'prefrontal_parietal': [0, 1, 2, 3, 6, 7, 10, 11, 16, 18],
 'prefrontal_occipital': [0, 1, 2, 3, 8, 9, 10, 11, 16],
 'central_temporal': [4, 5, 12, 13, 14, 15, 17],
 'central_parietal': [4, 5, 6, 7, 17, 18],
 'central_occipital': [4, 5, 8, 9, 17],
 'temporal_parietal': [6, 7, 12, 13, 14, 15, 18],
 'temporal_occipital': [8, 9, 12, 13, 14, 15],
 'parietal_occipital': [6, 7, 8, 9, 18],
 'prefrontal_central_temporal': [0,
  1,
  2,
  3,
  4,
  5,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17],
 'prefrontal_central_parietal': [0, 1, 2, 3, 4, 5, 6, 7, 10, 11, 16, 17, 18],
 'prefrontal_central_occipital': [0, 1, 2, 3, 4, 5, 8, 9, 10, 11, 16, 17],
 'prefrontal_temporal_parietal': [0,
  1,
  2,
  3,
  6,
  7,
  10,
  11,
  12,
  

In [28]:
partition = "temporal"
prefrontal_mean_nc = np.std(mean_nc[regions[partition]],axis=0)

prefrontal_mean_mci = np.std(mean_mci[regions[partition]],axis=0)

t_stat, p_value = stats.ttest_ind(prefrontal_mean_nc, prefrontal_mean_mci)
print("t:",t_stat,"\np:",p_value)

t: 1.5041448798203538 
p: 0.13848028951717986


In [17]:
import numpy as np
from scipy import stats

# 假设有两个群体的数据（每行一个样本，每列一个特征）
group_A = np.random.randn(30, 5)  # 群体A有30个样本，5个特征
group_B = np.random.randn(30, 5)  # 群体B有30个样本，5个特征

# 对每个特征进行独立样本t检验
for i in range(group_A.shape[1]):
    t_stat, p_value = stats.ttest_ind(group_A[:, i], group_B[:, i])
    print(f"Feature {i+1}: t-statistic = {t_stat:.3f}, p-value = {p_value:.3f}")


Feature 1: t-statistic = -0.166, p-value = 0.869
Feature 2: t-statistic = 0.610, p-value = 0.544
Feature 3: t-statistic = 0.883, p-value = 0.381
Feature 4: t-statistic = 0.056, p-value = 0.955
Feature 5: t-statistic = 1.160, p-value = 0.251
